In [1]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType , DoubleType , StringType , StructField , StructType , DecimalType
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler, Imputer, StringIndexer
from pyspark.ml.classification import NaiveBayes ,DecisionTreeClassifier , LinearSVC , RandomForestClassifier
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas

In [2]:
spark = SparkSession.builder.appName("Spark Merge").getOrCreate()

In [3]:
Mawischema = StructType([StructField("anomalyID", IntegerType() ,True),
StructField("srcIP", StringType() ,True),
StructField("srcPort", IntegerType() ,True),
StructField("dstIP", StringType() ,True),
StructField("dstPort", IntegerType() ,True),
StructField("taxonomy", StringType() ,True),
StructField("heuristic", DoubleType() ,True),
StructField("distance", IntegerType() ,True),
StructField("label", StringType() ,True),
StructField("nbDetectors", StringType() ,True)
])


In [4]:
ISCXschema = StructType([StructField('Flow ID',StringType() ,True),
StructField("Source IP", StringType() ,True),
StructField("Source Port", IntegerType() ,True),
StructField("Destination IP", StringType() ,True),
StructField("Destination Port", IntegerType() ,True),
StructField("Protocol", IntegerType() ,True),
StructField("Timestamp", StringType() ,True),
StructField("Flow Duration", IntegerType() ,True),
StructField("Total Fwd Packets", IntegerType() ,True),
StructField("Total Backward Packets", IntegerType() ,True),
StructField("Total Length of Fwd Packets", IntegerType() ,True),
StructField("Total Length of Bwd Packets", DecimalType() ,True),
StructField("Fwd Packet Length Max", IntegerType() ,True),
StructField("Fwd Packet Length Min", IntegerType() ,True),
StructField("Fwd Packet Length Mean", DoubleType() ,True),
StructField("Fwd Packet Length Std", DoubleType() ,True),
StructField("Bwd Packet Length Max", IntegerType() ,True),
StructField("Bwd Packet Length Min", IntegerType() ,True),
StructField("Bwd Packet Length Mean", DoubleType() ,True),
StructField("Bwd Packet Length Std", DoubleType() ,True),
StructField("Flow Bytes/s", DoubleType() ,True),
StructField("Flow Packets/s", DoubleType() ,True),
StructField("Flow IAT Mean", DoubleType() ,True),
StructField("Flow IAT Std", DecimalType() ,True),
StructField("Flow IAT Max", IntegerType() ,True),
StructField("Flow IAT Min", IntegerType() ,True),
StructField("Fwd IAT Total", DecimalType() ,True),
StructField("Fwd IAT Mean", DoubleType() ,True),
StructField("Fwd IAT Std", DoubleType() ,True),
StructField("Fwd IAT Max", DecimalType() ,True),
StructField("Fwd IAT Min", DecimalType() ,True),
StructField("Bwd IAT Total", DecimalType() ,True),
StructField("Bwd IAT Mean", DoubleType() ,True),
StructField("Bwd IAT Std", DoubleType() ,True),
StructField("Bwd IAT Max", DecimalType() ,True),
StructField("Bwd IAT Min", DecimalType() ,True),
StructField("Fwd PSH Flags", IntegerType() ,True),
StructField("Bwd PSH Flags", IntegerType() ,True),
StructField("Fwd URG Flags", IntegerType() ,True),
StructField("Bwd URG Flags", IntegerType() ,True),
StructField("Fwd Header Length40", IntegerType() ,True),
StructField("Bwd Header Length", IntegerType() ,True),
StructField("Fwd Packets/s", DoubleType() ,True),
StructField("Bwd Packets/s", DoubleType() ,True),
StructField("Min Packet Length", IntegerType() ,True),
StructField("Max Packet Length", IntegerType() ,True),
StructField("Packet Length Mean", DoubleType() ,True),
StructField("Packet Length Std", DoubleType() ,True),
StructField("Packet Length Variance", DoubleType() ,True),
StructField("FIN Flag Count", IntegerType() ,True),
StructField("SYN Flag Count", IntegerType() ,True),
StructField("RST Flag Count", IntegerType() ,True),
StructField("PSH Flag Count", IntegerType() ,True),
StructField("ACK Flag Count", IntegerType() ,True),
StructField("URG Flag Count", IntegerType() ,True),
StructField("CWE Flag Count", IntegerType() ,True),
StructField("ECE Flag Count", IntegerType() ,True),
StructField("Down/Up Ratio", IntegerType() ,True),
StructField("Average Packet Size", DoubleType() ,True),
StructField("Avg Fwd Segment Size", DoubleType() ,True),
StructField("Avg Bwd Segment Size", DoubleType() ,True),
StructField("Fwd Header Length61", IntegerType() ,True),
StructField("Fwd Avg Bytes/Bulk", IntegerType() ,True),
StructField("Fwd Avg Packets/Bulk", IntegerType() ,True),
StructField("Fwd Avg Bulk Rate", IntegerType() ,True),
StructField("Bwd Avg Bytes/Bulk", IntegerType() ,True),
StructField("Bwd Avg Packets/Bulk", IntegerType() ,True),
StructField("Bwd Avg Bulk Rate", IntegerType() ,True),
StructField("Subflow Fwd Packets", IntegerType() ,True),
StructField("Subflow Fwd Bytes", IntegerType() ,True),
StructField("Subflow Bwd Packets", IntegerType() ,True),
StructField("Subflow Bwd Bytes", IntegerType() ,True),
StructField("Init_Win_bytes_forward", IntegerType() ,True),
StructField("Init_Win_bytes_backward", IntegerType() ,True),
StructField("act_data_pkt_fwd", IntegerType() ,True),
StructField("min_seg_size_forward", IntegerType() ,True),
StructField("Active Mean", DoubleType() ,True),
StructField("Active Std", DoubleType() ,True),
StructField("Active Max", DecimalType() ,True),
StructField("Active Min", DecimalType() ,True),
StructField("Idle Mean", DoubleType() ,True),
StructField("Idle Std", DoubleType() ,True),
StructField("Idle Max", DecimalType() ,True),
StructField("Idle Min", DecimalType() ,True),
StructField("Label", StringType() ,True)])

In [5]:
Mawi = spark.read.csv('/Mawilab2017Sample.csv',schema=Mawischema,header=True).cache()

In [9]:
UNSW = spark.read.csv('/UNSW_NB15Sample.csv',inferSchema=True,header=True).cache()

In [10]:
ISCX = spark.read.csv('/ISCX2017Sample.csv',schema=ISCXschema,header=True).cache()

In [11]:
Kdd = spark.read.csv('/KDDCup99Sample.csv',inferSchema=True,header=True).cache()

In [12]:
#ISCX.groupBy('Protocol').count().show()

In [13]:
Mawi = Mawi.withColumnRenamed('srcIP' ,'Source IP')
Mawi = Mawi.withColumnRenamed('dstIP' ,'Destination IP')
Mawi = Mawi.withColumnRenamed('srcPort' ,'Source Port')
Mawi = Mawi.withColumnRenamed('dstPort' ,'Destination Port')

In [14]:
UNSW = UNSW.withColumnRenamed('dur' ,'duration')
UNSW = UNSW.withColumnRenamed('proto' ,'Protocol')
UNSW = UNSW.withColumnRenamed('ct_src_dport_ltm' ,'Source Port')
UNSW = UNSW.withColumnRenamed('ct_dst_sport_ltm' ,'Destination Port')
UNSW = UNSW.withColumnRenamed('sbytes' ,'Source Bytes')
UNSW = UNSW.withColumnRenamed('dbytes' ,'Destination Bytes')

In [15]:
Kdd = Kdd.withColumnRenamed('protocol_type' ,'Protocol')
Kdd = Kdd.withColumnRenamed('src_bytes' ,'Destination Bytes')
Kdd = Kdd.withColumnRenamed('dst_bytes' ,'Source Bytes')

In [16]:
#Kddpand = pandas.read_csv('C:/Users/Mounir Hafsa/Desktop/bases/Sample Join/KDDCup99Sample.csv')

In [17]:
#mawipand = mawipand.rename(columns={' label':'label',' srcIP':'srcIP',' srcPort':'srcPort',' dstIP':'dstIP',' dstPort':'dstPort',' taxonomy':'taxonomy',' heuristic':'heuristic',' distance':'distance',' nbDetectors':'nbDetectors'})

In [18]:
ISCX = ISCX.withColumnRenamed('Flow Duration','duration')

In [19]:
Kdd.columns

['duration',
 'Protocol',
 'service',
 'flag',
 'Destination Bytes',
 'Source Bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'label']

In [20]:
UN = UNSW.select(['duration',
 'Protocol',
 'service',
 'state',
 'spkts',
 'dpkts',
 'Source Bytes',
 'Destination Bytes',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'tcprtt',
 'synack',
 'ackdat',
 'smean',
 'dmean',
 'trans_depth',
 'response_body_len',
 'ct_srv_src',
 'ct_state_ttl',
 'ct_dst_ltm',
 'Source Port',
 'Destination Port',
 'ct_dst_src_ltm',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_flw_http_mthd',
 'ct_src_ltm',
 'ct_srv_dst',
 'is_sm_ips_ports',
 'attack_cat',
 'label'])

In [21]:
Mawi_Lab = Mawi.select(['Source IP',
 'Source Port',
 'Destination IP',
 'Destination Port',
 'taxonomy',
 'heuristic',
 'distance',
 'label'])

In [22]:
ISCX_IDS = ISCX.select(['Source IP',
 'Source Port',
 'Destination IP',
 'Destination Port',
 'Protocol',
 'duration',
 'Total Fwd Packets',
 'Total Backward Packets',
 'Total Length of Fwd Packets',
 'Total Length of Bwd Packets',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Total',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Total',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Length40',
 'Bwd Header Length',
 'Fwd Packets/s',
 'Bwd Packets/s',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'FIN Flag Count',
 'SYN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'ACK Flag Count',
 'URG Flag Count',
 'CWE Flag Count',
 'ECE Flag Count',
 'Down/Up Ratio',
 'Average Packet Size',
 'Avg Fwd Segment Size',
 'Avg Bwd Segment Size',
 'Fwd Header Length61',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate',
 'Subflow Fwd Packets',
 'Subflow Fwd Bytes',
 'Subflow Bwd Packets',
 'Subflow Bwd Bytes',
 'Init_Win_bytes_forward',
 'Init_Win_bytes_backward',
 'act_data_pkt_fwd',
 'min_seg_size_forward',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min',
 'Label'])

In [23]:
#Kdd.createTempView("kdd")
#Mawi.createTempView("mawi")

In [24]:
#Kdd_Mawi = spark.sql("select * FROM mawi LEFT JOIN kdd ON 'kdd.label' = 'mawi. label'")

In [25]:
#kddf = spark.sql("select * FROM kdd")
#mawif = spark.sql("select * FROM mawi")

UN , Mawi_Lab , ISCX_IDS , Kdd

 cond = [df.name == df3.name, df.age == df3.age]
 UN['label'] == Kdd['label'],UN['duration'] == Kdd['duration'],UN['Protocol'] == Kdd['Protocol'],UN['service'] == Kdd['service'],UN['Source Bytes'] == Kdd['Source Bytes'],UN['Destination Bytes'] == Kdd['Destination Bytes']
df.join(df3, cond, 'outer').select(df.name, df3.age).collect()

In [26]:
#cond = [UN['label'] == Kdd['label'] , UN['duration'] == Kdd['duration'] , UN['Protocol'] == Kdd['Protocol'] , UN['service'] == Kdd['service'] , UN['Source Bytes'] == Kdd['Source Bytes'] , UN['Destination Bytes'] == Kdd['Destination Bytes']

In [27]:
df_joined = Kdd.join(UN,['label', 'duration','Protocol','service','Source Bytes','Destination Bytes'],how='full_outer')

In [28]:
df_joined.columns

['label',
 'duration',
 'Protocol',
 'service',
 'Source Bytes',
 'Destination Bytes',
 'flag',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'state',
 'spkts',
 'dpkts',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'tcprtt',
 'synack',
 'ackdat',

In [29]:
#df_joined.write.parquet('/kdd_mawi_dataframe.parquet')

In [30]:
df_joined.count()

11051

In [31]:
#df_joined.groupBy('label').count().show()

In [32]:
df_kdmwun = df_joined.join(ISCX_IDS,['label' , 'duration' , 'Protocol' ,'Source Port','Destination Port'],how='full_outer')

In [33]:
df_kdmwun.printSchema()

root
 |-- label: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Source Port: integer (nullable = true)
 |-- Destination Port: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- Source Bytes: integer (nullable = true)
 |-- Destination Bytes: integer (nullable = true)
 |-- flag: string (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- lnum_compromised: integer (nullable = true)
 |-- lroot_shell: integer (nullable = true)
 |-- lsu_attempted: integer (nullable = true)
 |-- lnum_root: integer (nullable = true)
 |-- lnum_file_creations: integer (nullable = true)
 |-- lnum_shells: integer (nullable = true)
 |-- lnum_access_files: integer (nullable = true)
 |-- lnum_outbound_cmds: integer (nul

In [34]:
#df_kdmwun.count()

In [35]:
#df_kdmwun.groupBy('service').count().show()

In [36]:
df_full = df_kdmwun.join(Mawi_Lab,['label' , 'Destination IP' , 'Source IP' ,'Source Port','Destination Port'],how='full_outer')

In [37]:
df_full.columns

['label',
 'Destination IP',
 'Source IP',
 'Source Port',
 'Destination Port',
 'duration',
 'Protocol',
 'service',
 'Source Bytes',
 'Destination Bytes',
 'flag',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'state',
 'spkts',
 'dpkts',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',


In [38]:
#df_full.count()

In [39]:
#df_full.groupBy('label').count().show()

In [40]:
df_full.columns

['label',
 'Destination IP',
 'Source IP',
 'Source Port',
 'Destination Port',
 'duration',
 'Protocol',
 'service',
 'Source Bytes',
 'Destination Bytes',
 'flag',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'state',
 'spkts',
 'dpkts',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',


In [41]:
#df_full.groupBy('taxonomy').count().show(200)

In [42]:
#df_full = df_full.filter(df_f ull['service'].isNotNull())

In [43]:
df_full.count()

35040

In [44]:
df_full.dtypes

[('label', 'string'),
 ('Destination IP', 'string'),
 ('Source IP', 'string'),
 ('Source Port', 'int'),
 ('Destination Port', 'int'),
 ('duration', 'double'),
 ('Protocol', 'string'),
 ('service', 'string'),
 ('Source Bytes', 'int'),
 ('Destination Bytes', 'int'),
 ('flag', 'string'),
 ('land', 'int'),
 ('wrong_fragment', 'int'),
 ('urgent', 'int'),
 ('hot', 'int'),
 ('num_failed_logins', 'int'),
 ('logged_in', 'int'),
 ('lnum_compromised', 'int'),
 ('lroot_shell', 'int'),
 ('lsu_attempted', 'int'),
 ('lnum_root', 'int'),
 ('lnum_file_creations', 'int'),
 ('lnum_shells', 'int'),
 ('lnum_access_files', 'int'),
 ('lnum_outbound_cmds', 'int'),
 ('is_host_login', 'int'),
 ('is_guest_login', 'int'),
 ('count', 'int'),
 ('srv_count', 'int'),
 ('serror_rate', 'double'),
 ('srv_serror_rate', 'double'),
 ('rerror_rate', 'double'),
 ('srv_rerror_rate', 'double'),
 ('same_srv_rate', 'double'),
 ('diff_srv_rate', 'double'),
 ('srv_diff_host_rate', 'double'),
 ('dst_host_count', 'int'),
 ('dst_host

In [45]:
df_full = df_full.na.fill({'Source Port' :0,
'Destination Port' :0,
'duration' :0,
'Source Bytes' :0,
'Destination Bytes' :0,
'land' :0,
'wrong_fragment' :0,
'urgent' :0,
'hot' :0,
'num_failed_logins' :0,
'logged_in' :0,
'lnum_compromised' :0,
'lroot_shell' :0,
'lsu_attempted' :0,
'lnum_root' :0,
'lnum_file_creations' :0,
'lnum_shells' :0,
'lnum_access_files' :0,
'lnum_outbound_cmds' :0,
'is_host_login' :0,
'is_guest_login' :0,
'count' :0,
'srv_count' :0,
'serror_rate' :0,
'srv_serror_rate' :0,
'rerror_rate' :0,
'srv_rerror_rate' :0,
'same_srv_rate' :0,
'diff_srv_rate' :0,
'srv_diff_host_rate' :0,
'dst_host_count' :0,
'dst_host_srv_count' :0,
'dst_host_same_srv_rate' :0,
'dst_host_diff_srv_rate' :0,
'dst_host_same_src_port_rate' :0,
'dst_host_srv_diff_host_rate' :0,
'dst_host_serror_rate' :0,
'dst_host_srv_serror_rate' :0,
'dst_host_rerror_rate' :0,
'dst_host_srv_rerror_rate' :0,
'spkts' :0,
'dpkts' :0,
'rate' :0,
'sttl' :0,
'dttl' :0,
'sload' :0,
'dload' :0,
'sloss' :0,
'dloss' :0,
'sinpkt' :0,
'dinpkt' :0,
'sjit' :0,
'djit' :0,
'swin' :0,
'stcpb' :0,
'dtcpb' :0,
'dwin' :0,
'tcprtt' :0,
'synack' :0,
'ackdat' :0,
'smean' :0,
'dmean' :0,
'trans_depth' :0,
'response_body_len' :0,
'ct_srv_src' :0,
'ct_state_ttl' :0,
'ct_dst_ltm' :0,
'ct_dst_src_ltm' :0,
'is_ftp_login' :0,
'ct_ftp_cmd' :0,
'ct_flw_http_mthd' :0,
'ct_src_ltm' :0,
'ct_srv_dst' :0,
'is_sm_ips_ports' :0,
'Total Fwd Packets' :0,
'Total Backward Packets' :0,
'Total Length of Fwd Packets' :0,
'Total Length of Bwd Packets' :0,
'Fwd Packet Length Max' :0,
'Fwd Packet Length Min' :0,
'Fwd Packet Length Mean' :0,
'Fwd Packet Length Std' :0,
'Bwd Packet Length Max' :0,
'Bwd Packet Length Min' :0,
'Bwd Packet Length Mean' :0,
'Bwd Packet Length Std' :0,
'Flow Bytes/s' :0,
'Flow Packets/s' :0,
'Flow IAT Mean' :0,
'Flow IAT Std' :0,
'Flow IAT Max' :0,
'Flow IAT Min' :0,
'Fwd IAT Total' :0,
'Fwd IAT Mean' :0,
'Fwd IAT Std' :0,
'Fwd IAT Max' :0,
'Fwd IAT Min' :0,
'Bwd IAT Total' :0,
'Bwd IAT Mean' :0,
'Bwd IAT Std' :0,
'Bwd IAT Max' :0,
'Bwd IAT Min' :0,
'Fwd PSH Flags' :0,
'Bwd PSH Flags' :0,
'Fwd URG Flags' :0,
'Bwd URG Flags' :0,
'Fwd Header Length40' :0,
'Bwd Header Length' :0,
'Fwd Packets/s' :0,
'Bwd Packets/s' :0,
'Min Packet Length' :0,
'Max Packet Length' :0,
'Packet Length Mean' :0,
'Packet Length Std' :0,
'Packet Length Variance' :0,
'FIN Flag Count' :0,
'SYN Flag Count' :0,
'RST Flag Count' :0,
'PSH Flag Count' :0,
'ACK Flag Count' :0,
'URG Flag Count' :0,
'CWE Flag Count' :0,
'ECE Flag Count' :0,
'Down/Up Ratio' :0,
'Average Packet Size' :0,
'Avg Fwd Segment Size' :0,
'Avg Bwd Segment Size' :0,
'Fwd Header Length61' :0,
'Fwd Avg Bytes/Bulk' :0,
'Fwd Avg Packets/Bulk' :0,
'Fwd Avg Bulk Rate' :0,
'Bwd Avg Bytes/Bulk' :0,
'Bwd Avg Packets/Bulk' :0,
'Bwd Avg Bulk Rate' :0,
'Subflow Fwd Packets' :0,
'Subflow Fwd Bytes' :0,
'Subflow Bwd Packets' :0,
'Subflow Bwd Bytes' :0,
'Init_Win_bytes_forward' :0,
'Init_Win_bytes_backward' :0,
'act_data_pkt_fwd' :0,
'min_seg_size_forward' :0,
'Active Mean' :0,
'Active Std' :0,
'Active Max' :0,
'Active Min' :0,
'Idle Mean' :0,
'Idle Std' :0,
'Idle Max' :0,
'Idle Min' :0,
'heuristic' :0,
'distance' :0})

In [46]:
df_full = df_full.na.fill({'label': '0', 'service': 'null' ,'Protocol' : 'null','flag': 'null','Destination IP': 'null','Source IP': 'null','state': 'null','attack_cat': 'null','taxonomy': 'null'})

In [47]:
LabelIndexer = StringIndexer().setInputCol("label").setOutputCol("Label_ix")
LabelIndexer.setHandleInvalid("keep")

StringIndexer_45208742a5fcdb6ce1ed

In [48]:
TaxIndexer = StringIndexer(inputCol='taxonomy', outputCol='taxonomy_ix').setHandleInvalid("keep")
SrcIndexer = StringIndexer(inputCol='Source IP', outputCol='Source_IP_ix').setHandleInvalid("keep")
ProtoIndexer = StringIndexer(inputCol='Protocol', outputCol='Protocol_ix').setHandleInvalid("keep")
ServIndexer = StringIndexer(inputCol='service', outputCol='service_ix').setHandleInvalid("keep")
flagIndexer = StringIndexer(inputCol='flag', outputCol='flag_ix').setHandleInvalid("keep")
statIndexer = StringIndexer(inputCol='state', outputCol='state_ix').setHandleInvalid("keep")
atkIndexer = StringIndexer(inputCol='attack_cat', outputCol='attack_cat_ix').setHandleInvalid("keep")
destIndexer = StringIndexer(inputCol='Destination IP', outputCol='Destination_IP_ix').setHandleInvalid("keep")

In [53]:
output_fixed = LabelIndexer.fit(df_full).transform(df_full)

In [54]:
output_fixed1 = TaxIndexer.fit(output_fixed).transform(output_fixed)
output_fixed2 = SrcIndexer.fit(output_fixed1).transform(output_fixed1)
output_fixed3 = ProtoIndexer.fit(output_fixed2).transform(output_fixed2)
output_fixed4 = ServIndexer.fit(output_fixed3).transform(output_fixed3)
output_fixed5 = flagIndexer.fit(output_fixed4).transform(output_fixed4)
output_fixed6 = statIndexer.fit(output_fixed5).transform(output_fixed5)
output_fixed7 = atkIndexer.fit(output_fixed6).transform(output_fixed6)
output_fixed8 = destIndexer.fit(output_fixed7).transform(output_fixed7)

In [105]:
#output_fixed8.write.csv('/finalMerge_Indexed.csv',header=True)

df_full = StringIndexer(inputCol='label', outputCol='Label_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='Destination IP', outputCol='Destination_IP_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='Source IP', outputCol='Source_IP_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='Protocol', outputCol='Protocol_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='service', outputCol='service_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='flag', outputCol='flag_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='state', outputCol='state_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='attack_cat', outputCol='attack_cat_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")
df_full = StringIndexer(inputCol='taxonomy', outputCol='taxonomy_ix').fit(df_full).transform(df_full).setHandleInvalid("keep")

In [55]:
inputCols = ['Source Port',
'Destination Port',
'duration',
'Source Bytes',
'Destination Bytes',
'land',
'wrong_fragment',
'urgent',
'hot',
'num_failed_logins',
'logged_in',
'lnum_compromised',
'lroot_shell',
'lsu_attempted',
'lnum_root',
'lnum_file_creations',
'lnum_shells',
'lnum_access_files',
'lnum_outbound_cmds',
'is_host_login',
'is_guest_login',
'count',
'srv_count',
'serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate',
'same_srv_rate',
'diff_srv_rate',
'srv_diff_host_rate',
'dst_host_count',
'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'spkts',
'dpkts',
'rate',
'sttl',
'dttl',
'sload',
'dload',
'sloss',
'dloss',
'sinpkt',
'dinpkt',
'sjit',
'djit',
'swin',
'stcpb',
'dtcpb',
'dwin',
'tcprtt',
'synack',
'ackdat',
'smean',
'dmean',
'trans_depth',
'response_body_len',
'ct_srv_src',
'ct_state_ttl',
'ct_dst_ltm',
'ct_dst_src_ltm',
'is_ftp_login',
'ct_ftp_cmd',
'ct_flw_http_mthd',
'ct_src_ltm',
'ct_srv_dst',
'is_sm_ips_ports',
'Total Fwd Packets',
'Total Backward Packets',
'Total Length of Fwd Packets',
'Total Length of Bwd Packets',
'Fwd Packet Length Max',
'Fwd Packet Length Min',
'Fwd Packet Length Mean',
'Fwd Packet Length Std',
'Bwd Packet Length Max',
'Bwd Packet Length Min',
'Bwd Packet Length Mean',
'Bwd Packet Length Std',
'Flow Bytes/s',
'Flow Packets/s',
'Flow IAT Mean',
'Flow IAT Std',
'Flow IAT Max',
'Flow IAT Min',
'Fwd IAT Total',
'Fwd IAT Mean',
'Fwd IAT Std',
'Fwd IAT Max',
'Fwd IAT Min',
'Bwd IAT Total',
'Bwd IAT Mean',
'Bwd IAT Std',
'Bwd IAT Max',
'Bwd IAT Min',
'Fwd PSH Flags',
'Bwd PSH Flags',
'Fwd URG Flags',
'Bwd URG Flags',
'Fwd Header Length40',
'Bwd Header Length',
'Fwd Packets/s',
'Bwd Packets/s',
'Min Packet Length',
'Max Packet Length',
'Packet Length Mean',
'Packet Length Std',
'Packet Length Variance',
'FIN Flag Count',
'SYN Flag Count',
'RST Flag Count',
'PSH Flag Count',
'ACK Flag Count',
'URG Flag Count',
'CWE Flag Count',
'ECE Flag Count',
'Down/Up Ratio',
'Average Packet Size',
'Avg Fwd Segment Size',
'Avg Bwd Segment Size',
'Fwd Header Length61',
'Fwd Avg Bytes/Bulk',
'Fwd Avg Packets/Bulk',
'Fwd Avg Bulk Rate',
'Bwd Avg Bytes/Bulk',
'Bwd Avg Packets/Bulk',
'Bwd Avg Bulk Rate',
'Subflow Fwd Packets',
'Subflow Fwd Bytes',
'Subflow Bwd Packets',
'Subflow Bwd Bytes',
'Init_Win_bytes_forward',
'Init_Win_bytes_backward',
'act_data_pkt_fwd',
'min_seg_size_forward',
'Active Mean',
'Active Std',
'Active Max',
'Active Min',
'Idle Mean',
'Idle Std',
'Idle Max',
'Idle Min',
'heuristic',
'distance',
'taxonomy_ix',
'Source_IP_ix',
'Protocol_ix',
'service_ix',
'flag_ix',
'state_ix',
'attack_cat_ix',
'Destination_IP_ix']

In [56]:
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [57]:
output_fixed8.count()

35040

In [58]:
final_data = assembler.transform(output_fixed8).select('label','Label_ix','features')

In [145]:
#final_data.write.json('C:/Users/Mounir Hafsa/Desktop/bases/Sample Join/Final_Data_Features.json')

In [59]:
train_data,test_data = final_data.randomSplit([0.7,0.3],seed = 1000)
print("Nombre Instances dans TRAIN_DATA ",train_data.count())
train_data.cache()
print("Nombre Instances dans Test_DATA ",test_data.count())
test_data.cache()

Nombre Instances dans TRAIN_DATA  24492
Nombre Instances dans Test_DATA  10548


DataFrame[label: string, Label_ix: double, features: vector]

In [60]:
model_nb = NaiveBayes(labelCol='Label_ix', featuresCol='features')
model_dt = DecisionTreeClassifier(labelCol='Label_ix', featuresCol='features',maxBins=11000)
model_rf = RandomForestClassifier(labelCol='Label_ix', featuresCol='features',maxBins=11000)
model_svm = LinearSVC(labelCol='Label_ix', featuresCol='features')

In [61]:
pipeline = Pipeline(stages=[LabelIndexer,TaxIndexer,SrcIndexer,ProtoIndexer,ServIndexer,flagIndexer,statIndexer,atkIndexer,destIndexer, assembler, model_dt])

In [62]:
model = pipeline.fit(df_full)

In [69]:
model.save("/Joined_Model")

In [63]:
predictions = model.transform(df_full)

In [67]:
evaluator = MulticlassClassificationEvaluator(labelCol="Label_ix",predictionCol="prediction",metricName="accuracy")

In [68]:
evaluator.evaluate(predictions)

0.9998858447488584

In [124]:
print("Training ..")
#NB_Model = model_nb.fit(train_data)
print("NaiveBayes..DONE ..")

Training ..
NaiveBayes..DONE ..


In [148]:
print("Training ..")
Dt_Model = model_dt.fit(train_data)
print("Decision Tree..DONE ..")

Training ..
Decision Tree..DONE ..


In [149]:
print("Training ..")
Rf_Model = model_rf.fit(train_data)
print("Random Forest..DONE ..")

Training ..
Random Forest..DONE ..


In [123]:
print("Training ..")
#Svm_Model = model_svm.fit(train_data)
print("SVM..DONE ..")

Training ..
SVM..DONE ..


In [150]:
print ("Classifiying..")
Random_predict = Rf_Model.transform(test_data)
print ("C'est bon !")

Classifiying..
C'est bon !


In [151]:
print ("C'est bon !")
Decision_predict = Dt_Model.transform(test_data)
print ("Classifiying..")

C'est bon !
Classifiying..


In [152]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Label_ix", predictionCol="prediction", metricName="accuracy")

In [153]:
rfc_acc = acc_evaluator.evaluate(Random_predict)
print('A random forest had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))

A random forest had an accuracy of: 89.49%


In [154]:
dt_acc = acc_evaluator.evaluate(Decision_predict)
print('A Decision Tree had an accuracy of: {0:2.2f}%'.format(dt_acc*100))

A Decision Tree had an accuracy of: 100.00%


In [155]:
Decision_predict.printSchema()

root
 |-- label: string (nullable = false)
 |-- Label_ix: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [156]:
Decision_predict.select(['label','Label_ix','prediction']).show(50)

+----------+--------+----------+
|     label|Label_ix|prediction|
+----------+--------+----------+
|    BENIGN|     4.0|       4.0|
|    BENIGN|     4.0|       4.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
| anomalous|     1.0|       1.0|
|suspicious|     0.0|       0.0|
|suspicious|     0.0|       0.0|
|suspicious|     0.0|       0.0|
|suspicious|     0.0|       0.0|
|suspiciou

In [157]:
Decision_predict.write.json('/final_data_prediction.json')

In [159]:
train_data.unpersist()
test_data.unpersist()
df_full.unpersist()
df_kdmwun.unpersist()
df_joined.unpersist()

DataFrame[label: string, duration: double, Protocol: string, service: string, Source Bytes: int, Destination Bytes: int, flag: string, land: int, wrong_fragment: int, urgent: int, hot: int, num_failed_logins: int, logged_in: int, lnum_compromised: int, lroot_shell: int, lsu_attempted: int, lnum_root: int, lnum_file_creations: int, lnum_shells: int, lnum_access_files: int, lnum_outbound_cmds: int, is_host_login: int, is_guest_login: int, count: int, srv_count: int, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: int, dst_host_srv_count: int, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double, state: string, spkts: int, dpkts: int, rate: double, 

In [160]:
spark.stop()